In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Ejercicio 4: Modelo Probabilístico
Objetivo de la práctica
* Comprender los componentes del modelo vectorial mediante cálculos manuales y observación directa.
* Aplicar el modelo de espacio vectorial con TF-IDF para recuperar documentos relevantes.
* Comparar la recuperación con BM25 frente a TF-IDF.
* Analizar visualmente las diferencias entre los modelos.
* Evaluar si los rankings generados son consistentes con lo que considerarías documentos relevantes.

## Parte 0: Carga del Corpus

In [ ]:
import os

base_path = "/kaggle/input/20-newsgroups"
for folder in os.listdir(base_path):
    print(folder)


In [ ]:
import os

# Ruta a tu carpeta con los 20 archivos .txt
data_dir = "/kaggle/input/20-newsgroups"

# Crear una lista para almacenar todo el texto (el corpus)
corpus = []

# Recorrer los 20 archivos .txt
for fname in sorted(os.listdir(data_dir)):
    if fname.endswith(".txt"):
        path = os.path.join(data_dir, fname)
        with open(path, encoding="utf-8", errors="ignore") as f:
            text = f.read().strip()
            corpus.append(text)

print(f"Total documentos cargados: {len(corpus)}")



In [ ]:
corpus

## Parte 1: Cálculo de TF, DF, IDF y TF-IDF
Actividad
* Utiliza el corpus cargado.
* Construye la matriz de términos (TF), y calcula la frecuencia de documentos (DF)
* Calcula TF-IDF utilizando sklearn.
* Visualiza los valores en un DataFrame para analizar las diferencias entre los términos.

### Limpiar el Corpus

In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_text(text):
    # Minusculas
    text = text.lower()
    
    # Quitar puntuación y números
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Tokenizar
    tokens = text.split()
    
    # Quitar stopwords y lematizar
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    # Unir tokens nuevamente en string
    return " ".join(tokens)


In [ ]:
preprocessed_corpus = [preprocess_text(doc) for doc in corpus]

print("Documento original:")
print(corpus[0][:300])

print("\nDocumento preprocesado:")
print(preprocessed_corpus[0][:300])


### Crear la matriz de terminos TF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Inicializamos el vectorizador
vectorizer = CountVectorizer(stop_words='english')  # eliminamos stopwords básicas
X_tf = vectorizer.fit_transform(preprocessed_corpus)

# X_tf es una matriz sparse (20 documentos x n_terms)
print("Matriz TF:", X_tf.shape)


### Obtener la lista de terminos

In [ ]:
terms = vectorizer.get_feature_names_out()
print("Número de términos:", len(terms))
print("Primeros 10 términos:", terms[:10])

### Calcular la frecuencia de documentos DF

In [ ]:
# Convertimos la matriz TF a booleana: 1 si la palabra aparece, 0 si no
X_bool = (X_tf > 0).astype(int)

# Sumamos a lo largo de las filas para obtener DF
df_counts = np.array(X_bool.sum(axis=0)).flatten()

# Creamos un DataFrame para ver los resultados
import pandas as pd
df = pd.DataFrame({"term": terms, "DF": df_counts})
df = df.sort_values(by="DF", ascending=False)
df.head(10)

### Visualizar matriz TF-DF

In [ ]:
# Convertir matriz sparse a DataFrame (solo si quieres inspeccionarla)
tf_df = pd.DataFrame(X_tf.toarray(), columns=terms)
tf_df.head()


### Calcular matriz TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Inicializamos TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')  # opcional eliminar stopwords básicas

# Ajustamos y transformamos el corpus
X_tfidf = vectorizer.fit_transform(preprocessed_corpus)

# Obtenemos la lista de términos
terms = vectorizer.get_feature_names_out()


In [ ]:
print("Número de documentos:", X_tfidf.shape[0])
print("Número de términos:", X_tfidf.shape[1])
print("Primeros 10 términos:", terms[:10])


In [ ]:

tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=terms)
tfidf_df.head()
